Train validation data with convnet

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from modelval import pairptl, network, trainer, dataset
from modelval.ArbDataGen import arb_w_gen

% matplotlib inline
%load_ext autoreload
    
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Load data frame
data = pd.read_csv('/src/Plasticity_Ker/data/kernel_training_data_auto.csv')

In [17]:
# Visualize kernel
from modelval.kernel import KernelGen
ker_test = KernelGen()
# plt.plot(ker_test.bilat_ker, label='ker_pre')
# plt.plot(ker_test.unilat_ker, label='ker_post')
# plt.legend()

In [18]:
# Generate data
spk_pairs, targets = arb_w_gen(df=data, ptl_list=[1], kernel=ker_test, aug_times=1)

In [19]:
# Create the network
toy_data_net = network.PairNet(kernel=ker_test, kernel_pre=ker_test.bilat_ker, kernel_post=ker_test.unilat_ker, n_input=spk_pairs.shape[1])

In [20]:
# Create the trainer
toy_net_trainer = trainer.Trainer(toy_data_net.loss, input_name=toy_data_net.inputs, target_name=toy_data_net.target, save_dir='/src/Plasticity_Ker/model',  optimizer_config={'learning_rate': toy_data_net.lr})

In [21]:
# Obtain the training and validation data
from sklearn.model_selection import train_test_split

X_train, X_vali, y_train, y_vali = train_test_split(spk_pairs, targets, test_size=0.1)
train_data = dataset.Dataset(X_train, y_train)
vali_data = dataset.Dataset(X_vali, y_vali)

In [22]:
# Learn the kernel from random initialization
learning_rate = 0.001
iterations = 3
for i in range(iterations):
    # test_net.train(train_data, vali_data, feed_dict={lr: learning_rate, l1: 0.001, l2: 0.001})
    toy_net_trainer.train(train_data, vali_data, feed_dict={toy_data_net.lr: learning_rate})
    learning_rate = learning_rate/3


Initial validation cost=0.00000


KeyboardInterrupt: 